In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
from smount_predictors import SeamountTransformer
from smount_predictors import SeamountHelp
import plotly.express as px
import pandas as pd
import numpy as np

In [2]:
coords = SeamountHelp.readKMLbounds(Path('data') / 'Seamount_training_zone.kml')
training_data = SeamountHelp.readAndFilterGRD(Path('data') / 'vgg_swot.grd', coords[:2], coords[2:])

In [3]:
tran_inst = SeamountTransformer()
# tran_inst.sigma = 0.5
tran_inst.CONV_KERN = np.array([[1]])

In [4]:
numpy_train = training_data.to_dataframe().reset_index()[['lon', 'lat', 'z']].to_numpy()

In [5]:
trans = tran_inst.transform(numpy_train)
# trans = numpy_train[:, 2]

In [6]:
numpy_train[:, 2] = trans.reshape(-1)
numpy_train = numpy_train
px.scatter(x=numpy_train[:, 0], y=numpy_train[:, 1], color=numpy_train[:, 2]).update_layout(width=800, height=800, xaxis_title='lon', yaxis_title='lat', coloraxis_colorbar_title='z').show()